In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [0]:
path = get_file('nietzsche.txt',origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

In [0]:
with io.open(path,encoding='utf-8') as f:
  text = f.read().lower()

In [4]:
chars = sorted(list(set(text)))
print(chars)
print(len(chars))

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ä', 'æ', 'é', 'ë']
57


In [0]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i,c) for i,c in enumerate(chars))  

In [6]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0,len(text)-maxlen,step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb_sequences :',len(sentences))

nb_sequences : 200285


In [0]:
x = np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
y = np.zeros((len(sentences),len(chars)),dtype=np.bool)

In [0]:
for i,sentence in enumerate(sentences):
  for t,char in enumerate(sentence):
    x[i,t,char_indices[char]] = 1
  y[i,char_indices[next_chars[i]]] = 1

In [9]:
print('Building model...')
model = Sequential()
model.add(LSTM(128,input_shape=(maxlen,len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

Building model...


In [0]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [0]:
def sample(preds,temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds)/temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, logs):
  print()
  print('---Generating text after Epoch: %d' %epoch)
  start_index = random.randint(0,len(text)- maxlen - 1)
  for diversity in [0.2, 0.5, 1.0, 1.2]:
    print('----- diversity:', diversity)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
      x_pred = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.
      preds = model.predict(x_pred, verbose=0)[0]
      next_index = sample(preds, diversity)
      next_char = indices_char[next_index]
      generated += next_char
      sentence = sentence[1:] + next_char
      sys.stdout.write(next_char)
      sys.stdout.flush()
    print()
    
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
model.fit(x, y,batch_size=128,epochs=30,callbacks=[print_callback])

Epoch 1/30
200285/200285 [==============================] - 367s 2ms/step - loss: 3.2801



---Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "elf.

77. with his principles a man seek"
elf.

77. with his principles a man seekeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeheeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee

eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
----- diversity: 0.5
----- Generating with seed: "elf.

77. with his principles a man seek"
elf.

77. with his principles a man seekeceeeerei.eeie eheioeee .eeee ee.eaee.eeeeeeeeeeeeeieiee e leieeeee  eae eeeeeeeeee.eeeeeeeeee.eee eeeece eee eeeee.eeeeeeeeeee ee eee e;
ieeeieii e  eeee.eei  e eeeeieeeeieeeeeeeee eeae yeeeeiee e  e.eeeeee.eeee  eseeeeese eee  sneeeieheeeeei eneeeieibi eeeieeeeeeee

eeneee  e ieeeedeeeeeindee.eeeeeeee ee ee ie  eidhees eeeseeeii e a eeeae.e eenteeeee ee ee eee. .e.ei eieeeeeeeh eeeeeeeeehe.eneeeee
----- diversity: 1.0
----- Generating with seed: "elf.

77. with his principles a man seek"
elf.

77. with his principles a man seek.. eiahiesdeieetieeeiuwei.ethi fe  ecelehp emhdeiun.hevfviee . yieee  iee. t e eeisaesieideermedirhe.ereeen;eeeaeeyetowpi   ceeecdtc.eo e.ieiadie heeeg yidae .naeheuebuf  e.  eepgieeliei   
ve ,. deeb ea.eeheuscae evdideh . unvieneiscne

e en  u d.euiee e etbev thmc  p
eebdyie h.neoid
ie  ueepeai.el(eeeehtieee i ..aeceoue.teecadrfui bc eok.eluh.eefeve. ipecuh. w.oehi lheiae y uue ,aeiebmdcigeoeseiee
----- diversity: 1.2
----- Generating with seed: "elf.

77. with his principles a man seek"
elf.

77. with his principles a man seekuiesn siecyeohoeencneu  un
n te(ee baclepuh-ha.kanenrie.ts.eicaehtpey elvbedocc.ri ee eirenpthvifu;a.phheej;y
fd;o hsinu.,
esp  
dceeenubnai ueib.ees enw.
 . e
neee aneveeryeothe,ed
n n  ibi hhsenaseed,.es;.inebre

...

n n n nn nnnnnnnnnn nnr n nnnnnn  nnn nr  np  p nd nn n s nnnnnnn nn n nnn n nnn nnn n nnr nnn n nn   nnnnn nn n n nn nnnnns  nnnn nnr nnnn n nn  n nnnn  n nnn n n n  n n nnnn nnn nnr nn n nnnnnnnnnp  nnn n n n n nn nnn  n nn nn nn nn nnnnn nnnnnnnnnn nnnn nnnnnn  n n nnnnnn nnnn nnn nn nnnnnnnnnnnn nnnnnnnn nnnnnnnn nnnn n nnnnnnnnnnnnn n nn nn n nnnnnnnn n n nnnn  nnnn  nn n nn nnnnnn
----- diversity: 0.5
----- Generating with seed: "gs may not be lost on
his surface and fi"
gs may not be lost on
his surface and finannp nnn  n nm nn n nm  nn m o ns  nr nnn 

pus   nnoleus m s m   onnd n  nnnnar nnns   f ap  b nn ns nnl sonpunn nap m   nrod nn  r u-n up nd nn rh   nnhh nr   nnr a  n n u  nno  nm nhor   ns ven n p n n nr s  a na nn s r l de n nd p np nn  n an ns  d nn  ap n u ar nm  npennna ns n pens a s nnennnnp nnnnuv nr n nun  e p nm n unsonun ion nd e n nr b r np nna n s n sh nnonsann nn r ro n ros  np, nn 
----- diversity: 1.0
----- Generating with seed: "gs may not be lost on
his surface and fi"
gs may not be lost on
his surface and fiso la lon usp d pind rap sou mend  v  ondpewhepo r am o sonnpt ,    

 osn p  soan n, ta s npotitnan w n-mefa-u-um nvilsl  shhhipugir w nnngep ps-u pddh ncopp  vo opoaarodphn annl at bun n-qh s  ans" rs,or m ns ph np nc ran sr   vunphpanfh nnam n orunnlpuu p np ,ebonca inn m psch u  dh n nnnaw -p pr usab nnmyaon-mor oed v lbeulol rp apil omenpap  s p p"
osaugens  kivsem  so o  oop saf nntu w s na u 
----- diversity: 1.2
----- Generating with seed: "gs may not be lost on
his surface and fi"
gs may not be lost on
his surface and fi yor g-apo h  m m,  p dauph r"ap rpp  cuw c -prm nm vul n ncea-ui)aah caerannoiondos dh

 rp nsioc us m,osenm oosi  uopl nonh t pg--lnmab dorp n  a nailrp awelh
 bum raenaser l r ors-detpahhilhic r odoc pusruuse ninadahkshro-u gauvoa t kl unem duv r n"hare nnnmea mupp ri numucr. n"ils a ne l uo var,elna-efhi"n c upath  s noninud n -phrhy xorococrcfu"v  onpfir "oous od m f hopy u phe few n"c  nn, 
oi
Epoch 18/30
 15488/200285 [=>............................] - ETA: 5:01 - loss: 3.1260

136320/200285 [===================>..........] - ETA: 1:43 - loss: 3.1202